# Weekly demand utility

Import all the needed modules

In [1]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
sns.set(color_codes=True)

Import from the statistics file the columns needed

In [2]:
excel_file = 'Reported MO.xlsx'
df = pd.read_excel(excel_file)
df['Component no'] = df['Component no'].astype(str)
df.rename(columns={'Component no': 'Item no'}, inplace=True)
df.head()

,Product no,Product name,Work Center,MO no,Op no,Seq no,Date,Status,Item no,Component name,Reserved qty,Rep qty
0,65510030,GXK14-Droplet eliminator,645-060 PROFILE CUTTING & PUNCHING,1004352363,10,2,20190311,99,55144230,GX14-Top/bottom profile,4.0,4.0
1,65510030,GXK14-Droplet eliminator,645-060 PROFILE CUTTING & PUNCHING,1004352363,10,4,20190311,99,55144030,GX14-Side profile,4.0,4.0
2,65510030,GXK14-Droplet eliminator,645-060 PROFILE CUTTING & PUNCHING,1004352363,10,6,20190311,99,90722010,Profile for droplet catcher,33.6,33.6
3,65510030,GXK14-Droplet eliminator,645-060 PROFILE CUTTING & PUNCHING,1004352407,10,2,20190311,99,55144230,GX14-Top/bottom profile,4.0,4.0
4,65510030,GXK14-Droplet eliminator,645-060 PROFILE CUTTING & PUNCHING,1004352407,10,4,20190311,99,55144030,GX14-Side profile,4.0,4.0


Import from the items to buy

In [3]:
excel_file = 'Items to buy.xlsx'
df_buy = pd.read_excel(excel_file)
df_buy['Item no'] = df_buy['Item no'].astype(str)
df_buy.head()

,Item no,Item name
0,90652002,Perfil esquina estandar DV
1,90652005,Perfil esquina grande DV
2,90653500,Perfil central grande DV
3,90653512,Perfil rectangular DV
4,90653510,Perfil fuste DV


Get dates and number of week

In [4]:
df['Date'] = pd.to_datetime(df['Date'], format = '%Y%m%d')
df['Week'] = df['Date'].dt.week
df.head()

,Product no,Product name,Work Center,MO no,Op no,Seq no,Date,Status,Item no,Component name,Reserved qty,Rep qty,Week
0,65510030,GXK14-Droplet eliminator,645-060 PROFILE CUTTING & PUNCHING,1004352363,10,2,2019-03-11,99,55144230,GX14-Top/bottom profile,4.0,4.0,11
1,65510030,GXK14-Droplet eliminator,645-060 PROFILE CUTTING & PUNCHING,1004352363,10,4,2019-03-11,99,55144030,GX14-Side profile,4.0,4.0,11
2,65510030,GXK14-Droplet eliminator,645-060 PROFILE CUTTING & PUNCHING,1004352363,10,6,2019-03-11,99,90722010,Profile for droplet catcher,33.6,33.6,11
3,65510030,GXK14-Droplet eliminator,645-060 PROFILE CUTTING & PUNCHING,1004352407,10,2,2019-03-11,99,55144230,GX14-Top/bottom profile,4.0,4.0,11
4,65510030,GXK14-Droplet eliminator,645-060 PROFILE CUTTING & PUNCHING,1004352407,10,4,2019-03-11,99,55144030,GX14-Side profile,4.0,4.0,11


Create pivot table from the df

In [5]:
pivot_df = pd.pivot_table(df, values = 'Rep qty', index = 'Item no' , columns = 'Week', aggfunc=np.sum)
pivot_df = pivot_df.fillna(0)
pivot_df = pd.DataFrame(pivot_df.to_records())
pivot_df.head()

,Item no,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,0645000,33.0,99.0,33.0,0.0,0.0,33.0,0.0,0.0,0.0,...,0.0,0.0,231.0,33.0,33.0,0.0,66.0,33.0,0.0,0.0
1,0645001,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,39.0,39.0,0.0,0.0,0.0,39.0
2,0645002,45.0,45.0,0.0,0.0,0.0,45.0,45.0,0.0,0.0,...,0.0,0.0,0.0,0.0,45.0,45.0,0.0,0.0,0.0,0.0
3,0645003,0.0,0.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0,...,0.0,153.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0645004,116.0,0.0,116.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,58.0,58.0,58.0,0.0,0.0,0.0,0.0,0.0,58.0


Stats

In [6]:
col_len = len(pivot_df.columns)
pivot_df['Total'] = pivot_df.iloc[:, 1:col_len].sum(axis=1, numeric_only=True)
pivot_df['Mean'] = pivot_df.iloc[:, 1:col_len].mean(axis=1, numeric_only=True)
pivot_df['StD'] = pivot_df.iloc[:, 1:col_len].std(axis=1, numeric_only=True) 
pivot_df['CV'] = pivot_df['StD'] / pivot_df['Mean']
pivot_df['Min'] = pivot_df.iloc[:, 1:col_len].min(axis=1, numeric_only=True)
pivot_df['Q25'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.25, axis=1, numeric_only=True).round(0)
pivot_df['Median'] = pivot_df.iloc[:, 1:col_len].median(axis=1, numeric_only=True)
pivot_df['Q67'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.67, axis=1, numeric_only=True).round(0)
pivot_df['Q75'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.75, axis=1, numeric_only=True).round(0)
pivot_df['Max'] = pivot_df.iloc[:, 1:col_len].max(axis=1, numeric_only=True)
pivot_df['Monthly'] = 4.33*pivot_df['Q67']
pivot_df['Monthly'] = pivot_df['Monthly'].round(0)
pivot_df.head()

,Item no,1,2,3,4,5,6,7,8,9,...,Mean,StD,CV,Min,Q25,Median,Q67,Q75,Max,Monthly
0,0645000,33.0,99.0,33.0,0.0,0.0,33.0,0.0,0.0,0.0,...,19.038462,37.736944,1.982142,0.0,0.0,0.0,33.0,33.0,231.0,143.0
1,0645001,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28.500000,45.109670,1.582795,0.0,0.0,0.0,39.0,39.0,195.0,169.0
2,0645002,45.0,45.0,0.0,0.0,0.0,45.0,45.0,0.0,0.0,...,26.826923,55.120446,2.054669,0.0,0.0,0.0,8.0,45.0,270.0,35.0
3,0645003,0.0,0.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0,...,30.403846,50.759046,1.669494,0.0,0.0,0.0,9.0,51.0,153.0,39.0
4,0645004,116.0,0.0,116.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.230769,54.434928,1.742990,0.0,0.0,0.0,10.0,58.0,232.0,43.0


Merge datasets

In [7]:
pivot_df = pd.merge(df_buy, pivot_df, on='Item no')
pivot_df.head()

,Item no,Item name,1,2,3,4,5,6,7,8,...,Mean,StD,CV,Min,Q25,Median,Q67,Q75,Max,Monthly
0,90652002,Perfil esquina estandar DV,1062.787,1227.180,1608.346,934.782,147.162,880.320,1158.534,363.820,...,689.472692,374.351754,0.542954,0.0,401.0,663.8600,832.0,942.0,1650.490,3603.0
1,90652005,Perfil esquina grande DV,0.000,236.810,424.832,279.770,0.000,333.194,567.196,133.966,...,138.152500,172.858474,1.251215,0.0,0.0,72.8295,166.0,239.0,829.873,719.0
2,90653500,Perfil central grande DV,10.794,131.608,166.394,98.386,0.000,138.017,421.066,81.810,...,70.604000,85.115454,1.205533,0.0,2.0,46.8885,82.0,103.0,421.066,355.0
3,90653512,Perfil rectangular DV,196.782,334.834,302.583,200.354,54.233,210.134,284.658,79.433,...,149.089404,96.708539,0.648661,0.0,75.0,126.8120,196.0,203.0,432.508,849.0
4,90653510,Perfil fuste DV,153.728,287.302,502.975,208.041,52.166,301.020,540.834,156.884,...,193.123692,131.273498,0.679738,0.0,93.0,157.4005,230.0,276.0,540.834,996.0


Create histograms

In [8]:
columns = pivot_df.columns[2:54]
total_items = len(pivot_df['Item no'])

for item in range(total_items):
    item_no = pivot_df.loc[item]['Item no']
    item_name = pivot_df.loc[item]['Item name']
    item_q = pivot_df.loc[item]['Q67']
    max_qty = pivot_df.loc[item]['Max']
    sns.distplot(pivot_df.loc[item, columns], hist_kws={'cumulative': True}, kde_kws={'cumulative': True}, bins=20)
    plt.xlabel('Qty')
    plt.ylabel('Weekly Probability')
    plt.title('Item ' + item_no + ' - ' + item_name)
    plt.xlim(left=0, right=max_qty)
    plt.grid(True)
    plt.axvline(x=item_q, color='g')
    plt.text(item_q, 0.25, ' Q67 = '+str(item_q))
    plt.savefig('Items/'+item_no+'.png', dpi=100)
    plt.clf()
    
print('Done bro!')

Done bro!


<Figure size 432x288 with 0 Axes>

In [9]:
pivot_df

,Item no,Item name,1,2,3,4,5,6,7,8,...,Mean,StD,CV,Min,Q25,Median,Q67,Q75,Max,Monthly
0,90652002,Perfil esquina estandar DV,1062.787,1227.180,1608.346,934.782,147.162,880.320,1158.534,363.820,...,689.472692,374.351754,0.542954,0.0,401.0,663.8600,832.0,942.0,1650.490,3603.0
1,90652005,Perfil esquina grande DV,0.000,236.810,424.832,279.770,0.000,333.194,567.196,133.966,...,138.152500,172.858474,1.251215,0.0,0.0,72.8295,166.0,239.0,829.873,719.0
2,90653500,Perfil central grande DV,10.794,131.608,166.394,98.386,0.000,138.017,421.066,81.810,...,70.604000,85.115454,1.205533,0.0,2.0,46.8885,82.0,103.0,421.066,355.0
3,90653512,Perfil rectangular DV,196.782,334.834,302.583,200.354,54.233,210.134,284.658,79.433,...,149.089404,96.708539,0.648661,0.0,75.0,126.8120,196.0,203.0,432.508,849.0
4,90653510,Perfil fuste DV,153.728,287.302,502.975,208.041,52.166,301.020,540.834,156.884,...,193.123692,131.273498,0.679738,0.0,93.0,157.4005,230.0,276.0,540.834,996.0
5,90655000,Perfil Superior Ral7024,185.300,203.000,209.000,56.000,0.000,0.000,126.000,25.000,...,72.897654,65.339686,0.896321,0.0,26.0,54.9000,87.0,100.0,277.400,377.0
6,90655001,Perfil inferior Ral7024,118.900,141.000,144.000,36.000,0.000,0.000,84.000,20.000,...,47.220192,42.658551,0.903396,0.0,19.0,35.4000,55.0,65.0,186.600,238.0
7,90655003,2 Mullion (Perfil rectangular con rebaje),16.800,108.000,106.000,0.000,0.000,0.000,123.000,25.000,...,55.585192,76.147599,1.369926,0.0,11.0,33.5000,51.0,75.0,388.512,221.0
8,90655002,Mullion (Perfil rectangular),118.100,180.000,201.000,29.000,0.000,0.000,143.000,15.000,...,69.374923,77.977324,1.123999,0.0,18.0,42.5200,76.0,94.0,385.708,329.0
9,90655040,Perfil PVC Panel,778.400,725.000,888.000,157.000,0.000,0.000,559.100,300.500,...,332.426923,326.716582,0.982822,0.0,113.0,242.7500,380.0,405.0,1657.000,1645.0


Output to Excel

In [10]:
name = 'Weekly demand based on MOs.xlsx'
writer = pd.ExcelWriter(name)
pivot_df.to_excel(writer, index = False)
writer.save()
print('Done bro!')

Done bro!
